## Convert THEx Data to HDF5
1. load CSVs

In [1]:
import pandas as pd
# 
#  Data Note
#  The following data comes from an initialized THEx MultiModel. 
#  So, it represents all rows that have valid values for the mags and colors
#  
dpath = "/Users/marina/Documents/PhD/research/astro_research/data/testing/"
# Load CSVs of X and y
X = pd.read_csv(dpath + "X.csv")
y = pd.read_csv(dpath + "y.csv")

## Save to HDF5 with same format as WINE dataset
Hierarhcy:
- all
- folds
    - 1
        - tests
            - 1
        - training
            - 1
            - 2
            - ...
            - 8
            - 9
    - 2
    - ...
    - 9


Usage:

- Use folds/1/training/ 1 - 8 as training
- Use folds/1/training/9 as validation
- Use folds/1/testing/1 as testing


In [2]:
# Save X of certain class to HDF5 File
class_name = "Unspecified Ia"
class_indices = y.loc[y['transient_type'].str.contains(class_name)].index
class_X = X.iloc[class_indices]

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(class_X):
    print(train_index)

ImportError: No module named sklearn.model_selection

In [9]:
import pandas as pd

store = pd.HDFStore('store.h5')
store.root
store['df1']=class_X
store['group/df1']=class_X


In [ ]:
X.to_hdf(dpath+'all_thex_data.hdf5', key='thex_X', mode='w')

X.to_hdf(dpath + class_name.replace(" ", "_") + '_X.hdf5', key='thex_X', mode='w')

Make sure BigDataset works with this HDF5 File

In [ ]:
# Convert use h5py
import numpy as np
import h5py

csv_data = np.genfromtxt(dpath + "X.csv", delimiter=',', dtype=None, names=True, encoding=None)
print (csv_data.dtype)

# with h5py.File('SO_57120995.h5','w') as h5f :

#     for row in csv_data:   
#         date=row[0]
#         grp = h5f.require_group(date)

#         firm=row[1]
#     # convert row data to get list of all valuei entries
#         row_data=row.item()[2:]
#         h5f[date].create_dataset(firm,data=row_data)

In [ ]:
import h5py
f = h5py.File(dpath + "Unspecified_Ia_X.hdf5", "r")

## Wine Dataset Study

Outcomes

1. 159 samples in each training fold
2. 168 in validation 
3. 159 in testing

There is OVERLAP among all 3. So testing data inside of training and validation.
46/159 testing samples in the whole training set
9/159 testing samples in validation 

And overlap in training/validation.

wine dataset HDF5 data has the following hierarchy:
- all
- folds
    - 1
        - tests
            - 1
        - training
            - 1
            - 2
            - ...
            - 8
            - 9
    - 2
    - ...
    - 9

### Examine if there is any data overlap in training/validation/testing

In [ ]:
import sys
sys.path
sys.path.append('buml') 
import os
import Data.utils
os.environ["DATASETSPATH"]=""
data_source = "red_wine.hdf5"
training = "/folds/1/training/(1|2|3|4|5|6|7|8)"
dataset_file = os.path.join(os.environ["DATASETSPATH"], data_source)
training_dataset = Data.BigDataset(dataset_file, training, "data")
testing_dataset = Data.BigDataset(dataset_file, "/folds/1/tests/.*", "data")
validation_dataset = Data.BigDataset(dataset_file, "/folds/1/training/9", "data")

In [190]:
# Make sure no testing/training/validation overlaps
training_indices= [0,1,2,3,4,5,6,7]
validation_index = [8]

test_overlap_count = 0
for i in range(testing_fold.shape[0]):
    test_sample = testing_fold[i]
    # No overlap with training/testing
    total_training = 0 #to keep track of later.
    for ti in training_indices:
        training_fold = training_dataset.get_file(element = 0, index= ti)
        for row in training_fold:
            total_training +=1
            if (row == test_sample).all():
                test_overlap_count+=1
print("Overlap in testing and training " + str(test_overlap_count))

Overlap in testing and training 46


In [192]:
# No overlap with validation/testing
validation_fold = validation_dataset.get_file(element = 0, index= 0)
val_overlap_count = 0
for i in range(testing_fold.shape[0]):
    test_sample = testing_fold[i]
    for row in validation_fold:
        if (row == test_sample).all():
            val_overlap_count+=1
print("Overlap in validation and testing " + str(val_overlap_count))

Overlap in validation and testing 9


In [193]:
# No overlap with training/validation
matching_rows=0
for i in range(validation_fold.shape[0]):
    validation_sample = validation_fold[i]
    for ti in training_indices:
        training_fold = training_dataset.get_file(element = 0, index= ti)
        for row in training_fold:
            if (row == validation_sample).all():
                matching_rows +=1
print("Number of matching rows in validation + training " + str(matching_rows))


Number of matching rows in validation + training 41


In [ ]:
print("Total rows in training " + str(total_training))
print("Total rows in validation " + str(validation_fold.shape[0]))
print("Total rows in testing " + str(testing_fold.shape[0]))

### Manually pulling down wine-dataset and manually examining entries 

In [ ]:
wine_f = h5py.File("red_wine.hdf5", "r")
# wine_f["/"]

element_names="data"
element_names = element_names if isinstance(element_names, tuple) else (element_names,)

entries_regexp = "/folds/1/training/(1|2|3|4|5|6|7|8)" # Training 
# entries_regexp="/folds/1/tests/.*"  # Testing 
# entries_regexp = "/folds/1/training/9"  # Validation 
pats = entries_regexp.split("/")
pats.remove("")

"""
Coming up with entries:

We pass in "/folds/1/training/(1|2|3|4|5|6|7|8)" as the 'entries_regexp' 
so we are saying to use folds 1-8 for training

So, it selects the parts of the HDF5 dataset that are located at the part of the hierarhcy we
are selecting, using the regexp. 

- Use folds/1/training/ 1 - 8 as training
- Use folds/1/training/9 as validation
- Use folds/1/testing/1 as testing

""" 
import re
entries = [wine_f["/"]] 
for p in pats:
    new_entries = []
    for r in entries:
        for k, v in r.items(): 
            # Seeing if value of this HDF5 is one of the desired patterns
            if re.match("^%s$" % p, str(k)):
                new_entries.append(v)
    entries = new_entries 


#### The above does the exact same thing as:
# entries = [wine_f["/"]]
# for p in pats:
#     entries = [v for r in entries for k,
#                v in r.items() if re.match("^%s$" % p, str(k))]

## Match HDF5 Data to Wine dataset on Kaggle site
This is to ensure these are the same dataset, and they do appear to be. 

OUTCOME:

Data comes from wine dataset from Kaggle
Input variables (based on physicochemical tests):
1. fixed acidity (tartaric acid - g / dm^3)
2. volatile acidity (acetic acid - g / dm^3)
3. citric acid (g / dm^3)
4. residual sugar (g / dm^3)
5. chlorides (sodium chloride - g / dm^3
6. free sulfur dioxide (mg / dm^3)
7. total sulfur dioxide (mg / dm^3)
8. density (g / cm^3)
9. pH
10. sulphates (potassium sulphate - g / dm3)
11. alcohol (% by volume)

Output variable (based on sensory data): 
1. quality (score between 0 and 10)

In [ ]:
import sys
sys.path
sys.path.append('buml') 
import os
import Data.utils
os.environ["DATASETSPATH"]=""
data_source = "red_wine.hdf5"
training = "/folds/1/training/(1|2|3|4|5|6|7|8)"
dataset_file = os.path.join(os.environ["DATASETSPATH"], data_source)
training_dataset = Data.BigDataset(dataset_file, training, "data")

In [ ]:
# Get sample row and see if it is in other dataset
training_fold_0 = training_dataset.get_file(element = 0, index= 0)
sample_row = training_fold_0[0]
sample_row.tolist()

In [ ]:
import pandas as pd
wine_test_path = "~/Documents/PhD/research/astro_research/data/testing/wineQualityReds.csv"
wine_dataset = pd.read_csv(wine_test_path)

train_wine_dataset = wine_dataset.drop(columns=['Unnamed: 0', 'quality'])

In [ ]:
import numpy as np
for index, row in train_wine_dataset.iterrows():
    
    rtol = 1e-05
    atol = 1e-08
    res = np.allclose(sample_row, row, rtol, atol) 
    if res:
        print(row)
        print("Row index match: " + str(index))
        
        

In [ ]:
# options, args =parser.parse_args(["--theano", 
#                                   "--form", "MoG", 
#                                   "--dataset", "red_wine.hdf5", 
#                                   "--training_route", "/folds/1/training/(1|2|3|4|5|6|7|8)",
#                                  "--validation_route", "/folds/1/training/9",
#                                  "--test_route", "/folds/1/tests/.*",
#                                  "--samples_name", "data",
#                                  "--hlayers", "2", # 2 hidden layers
#                                   "--layerwise",
#                                   "--lr", "0.02",
#                                   "--wd", "0.02",
#                                   "--n_components", "10",
#                                   "--epoch_size", "100",
#                                   "--momentum", "0.9",
#                                   "--units", "100",
#                                   "--pretraining_epochs", "5",
#                                   "--validation_loops", "20",
#                                   "--epochs", "20",
#                                   "--normalize",
#                                   "--batch_size", "100",
#                                   "--show_training_stop", "red_wine"])
 